# Purpose
We are using GEE to calculate zonal statistics of EVI and LAI over the corn in California. We will masking them by a CDL corn layer but this could be a crop mask and exporting the data as a table.

The unique zonal statistic for this task, using EVI as the example, is as follows:

\begin{align}
 \frac{∑(EVI * Mask)}{∑(Mask)}
    \end{align}


Typical run me first.

In [ ]:
# Import ee
import ee

# Trigger the authentication flow.
ee.Authenticate()

# If tagging work
# ee.data.setDefaultWorkloadTag('') # name the process in the cell here, for best results put in each cell

# Imports
Bring in some basic imports needed.

## Geemap
If you are wanting an interactive map, this needs to be done with every notebook and every time the notebook has sat offline for a while.

In [ ]:
pip install geemap

## Folium
These maps are not truely interactive (i.e. can't inspect pixel values) but these are the most stable maps.

In [ ]:
# Import folium for viewing maps
import os
import folium
import geemap
from datetime import datetime
from IPython.display import Image
#geemap.update_package()
# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'

# Initialize the library.
ee.Initialize(project='') # put your gee project here!!!

print('Folium version: ' + folium.__version__)

## Displaying EE layers
This function is similar to the one that Google presented in the tutorials but it adds the functionality of displaying image collections, geometries, and feature collections. This is if you are planning to use the folium map interface.

In [ ]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    try:
        # Display ee.Image()
        if isinstance(ee_object, ee.image.Image):
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # Display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # Display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # Display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

    except:
        print("Could not display {}".format(name))

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


# Date variables
Here are the date variables that are going to be used in the experiment. If bringing in a new satellite source, enter the satellites start date here.

In [ ]:
# Satellite start dates
mydSTART = '2002-07-04' # Aqua
Today = datetime.today()

yearStart = 2022 # 2003
yearEnd   = 2022 # 2023

dateStart = 121 # 121 - May 1
dateEnd   = 160 # 288 - October 15

format = 'DDD'

dateStartD = ee.Date.parse(format, str(dateStart)).update(yearStart)
dateEndD = ee.Date.parse(format, str(dateEnd)).update(yearEnd)


# Admintrative layers
Add in some proprietary or open source admin layers.

In [ ]:
# Create a folium/gee map object.
my_map = folium.Map(location=[34,265], zoom_start=5, height=1000) # Location is [lat,long]
Mapa = geemap.Map(center=[34,265], zoom=5, height=1000)

# Admin layer
coun = ee.FeatureCollection("WM/geoLab/geoBoundaries/600/ADM0").filter(ee.Filter.eq('shapeName','United States'))
prov = ee.FeatureCollection("WM/geoLab/geoBoundaries/600/ADM1").filter(ee.Filter.eq('shapeName','California'))
dist = ee.FeatureCollection("WM/geoLab/geoBoundaries/600/ADM2").filter(ee.Filter.eq('shapeName','Santa Barbara'))

multiStateList=ee.List(['Iowa', 'Kansas', 'Texas'])
def filterMulti(ids):
    return ee.FeatureCollection(ids.map(lambda x: ee.FeatureCollection("WM/geoLab/geoBoundaries/600/ADM1").filterMetadata('shapeName','equals', x))).flatten()
multiReg = filterMulti(multiStateList)

# Viewing the imagery
#my_map.add_ee_layer(coun,{},'Country')
#my_map.add_ee_layer(prov,{},'Province')
#my_map.add_ee_layer(dist,{},'District')
#my_map.add_ee_layer(multiReg,{},'Multiple Regions')
#my_map.add_child(folium.LayerControl())
#display(my_map)

# Adding the imagery to the interactive panel
Mapa.addLayer(coun,{},'Country')
Mapa.addLayer(multiReg,{},'Multiple Regions')
Mapa.addLayer(prov,{},'Province')
Mapa.addLayer(dist,{},'District')
Mapa.addLayerControl()
Mapa

# CDL Reference

In [ ]:
Mapb = geemap.Map(center=[34,265], zoom=5, height=1000)

refYear = 2022

cdlyear = str(refYear)

# Select the CDL based on year
cdl_20xx = ee.Image(ee.ImageCollection("USDA/NASS/CDL").filter(ee.Filter.eq('system:index', cdlyear)).first()).select('cropland').clip(prov)
#print(cdl_20xx)

# Extract specific crops
corn = cdl_20xx.eq(1).updateMask(cdl_20xx.eq(1)).toFloat().clip(prov)
soy = cdl_20xx.eq(5).updateMask(cdl_20xx.eq(5)).toFloat()
cotton = cdl_20xx.eq(2).updateMask(cdl_20xx.eq(2)).toFloat()
rice = cdl_20xx.eq(3).updateMask(cdl_20xx.eq(3)).toFloat()
sorghum = cdl_20xx.eq(4).updateMask(cdl_20xx.eq(4)).toFloat()
dwheat = cdl_20xx.eq(22).updateMask(cdl_20xx.eq(22)).toFloat()
swheat = cdl_20xx.eq(23).updateMask(cdl_20xx.eq(23)).toFloat()
wwheat = cdl_20xx.eq(24).updateMask(cdl_20xx.eq(24)).toFloat()
sugarc = cdl_20xx.eq(25).updateMask(cdl_20xx.eq(25)).toFloat()

# Add to map
#Mapb.addLayer(cdl_20xx,{},'CDL')
Mapb.addLayer(corn,{'palette':'ffd400'},'Corn')
Mapb.addLayer(soy,{'palette':'267000'},'Soy')
Mapb.addLayerControl()
Mapb

# Load satellite imagery
Here we load AQUA EVI and LAI but any environmental source can be loaded.

In [ ]:
# Load MYD13A1.061: Aqua Vegetation Indices 16-Day Global 500m
myd13 = ee.ImageCollection("MODIS/061/MYD13A1").filterDate(mydSTART, Today)

# Load MYD15A2H.061: Aqua Leaf Area Index/FPAR 8-Day Global 500m
myd15 = ee.ImageCollection("MODIS/061/MYD15A2H").filterDate(mydSTART, Today)

# View/process satellite imagery
This section is dedicated to visualizing the imagery as well as creating the variables needed for the zonal statistics section. If you are going to bring in a new source just copy and paste one of the code blocks and change the variables as appropriate.

## View Aqua MODIS EVI

In [ ]:
# Create a folium/gee map object.
my_map = folium.Map(location=[34,265], zoom_start=5, height=1000) # Location is [lat,long]
Map8a = geemap.Map(center=[34,265], zoom=5, height=1000)

############Aqua MODIS MYD13 EVI############
def findEvi(i):
  iveg = i.select('EVI').multiply(0.0001)
  return iveg

eviFun = myd13.filterDate(dateStartD, dateEndD).map(findEvi)
eviFunMed = ee.ImageCollection(eviFun).median()

# Viewing the imagery
#my_map.add_ee_layer(eviFunMed.clip(multiReg),{'palette':['ff0000','ffff00','00ff00']},'EVI median')
#my_map.add_ee_layer(eviFun,{'min':0.19,'max':0.32},'EVI raw')
#my_map.add_child(folium.LayerControl())
#display(my_map)

# Adding the imagery to the interactive panel
Map8a.addLayer(eviFunMed.clip(prov),{'min':0,'max':0.9,'palette':['ff0000','ffff00','00ff00']},'EVI median')
#Map8a.addLayer(eviFun,{'min':0.19,'max':0.32},'EVI raw')
Map8a.addLayerControl()
Map8a

### Calculate EVI on Crop Mask

In [ ]:
# Create a folium/gee map object.
my_map = folium.Map(location=[34,265], zoom_start=5, height=1000) # location is [lat,long]
Map8aa = geemap.Map(center=[34,265], zoom=5, height=1000)

############Crop mask calculations############
mask = ee.Image(corn) # Crop mask here or CDL crop layer
maskFloat = mask.divide(100) # convert crop mask to float

def findEviOnMask(i):
  evi = i.select('EVI').multiply(0.0001) # extract the correct layer and apply the scaling factor
  eviOnMa = evi.multiply(maskFloat)

  joinFilter = ee.Filter.equals(
  leftField='shapeID',
  rightField='shapeID'
  )

  # Get summed EVI * floating mask per region per date
  summedEOnM = eviOnMa.reduceRegions(
      collection=prov, # add the shapefile here
      reducer=ee.Reducer.sum(),
      scale=100
  ).select(**{
    'propertySelectors': ['shapeGroup','shapeName','shapeID','sum'],
    'newProperties': ['shapeGroup','shapeName','shapeID','sumEOnM']
    })

  # Get summed floating mask per region per date
  summedMask = maskFloat.reduceRegions(
      collection=prov, # add the shapefile here
      reducer=ee.Reducer.sum(),
      scale=100
  ).select(**{
    'propertySelectors': ['shapeID','sum'],
    'newProperties': ['shapeID','sumMask']
    })

  joinHere = ee.Join.inner()
  joinedJoin = joinHere.apply(summedEOnM,summedMask,joinFilter)
  def calcDone(i):
      getProp = ee.Feature(i.get('primary')).copyProperties(i.get('secondary'))
      return getProp.set({'calcedVal': ee.Number(getProp.get('sumEOnM')).divide(ee.Number(getProp.get('sumMask')))}) # divide the sums
  calculatedDistricts = joinedJoin.map(calcDone)

  return calculatedDistricts

eviOnMaFun = myd13.filterDate(dateStartD, dateEndD).map(findEviOnMask).flatten()
eviOnMaFunMed = ee.ImageCollection(eviOnMaFun).median()

# Viewing the imagery
#my_map.add_ee_layer(eviFunMed.clip(multiReg),{'palette':['ff0000','ffff00','00ff00']},'EVI median')
#my_map.add_ee_layer(eviFun,{'min':0.19,'max':0.32},'EVI raw')
#my_map.add_child(folium.LayerControl())
#display(my_map)

# Adding the imagery to the interactive panel
#Map8aa.addLayer(mask.clip(multiReg),{'min':0,'max':100,},'Crop mask')
#Map8aa.addLayer(maskFloat.clip(multiReg),{'min':0,'max':1,},'Crop mask Float')
#Map8aa.addLayer(eviFunMed.clip(multiReg),{'min':0,'max':1},'EVI Median')
Map8aa.addLayer(eviOnMaFun,{},'EVI on Mask')
Map8aa.addLayerControl()
Map8aa

#print(eviOnMaFun.getInfo())

#### Export EVI on Crop Mask

In [ ]:
# Export the statistics
taskExE = ee.batch.Export.table.toDrive(**{
    'collection':eviOnMaFun,
    'folder':'Zonal_Stats', # you need to put the name of a folder from your Google Drive here
    'description':'EVIonCorn_season_2022', # name of the exported file
    'fileFormat':'CSV',
    'selectors':['system:index','shapeGroup','shapeName','shapeID','sumEOnM','sumMask','calcedVal'] # columns
})

taskExE.start()

In [ ]:
# Check the task status
task_status = taskExE.status()

# If the task is running
if task_status["state"] == "RUNNING":
  start_time = task_status["start_timestamp_ms"]/(60000)
  update_time = task_status['update_timestamp_ms']/(60000)
  elapsed_time = update_time-(start_time)
  hours, minutes = divmod(elapsed_time, 60)
  # Print a message saying that the task is running and how long it has been running
  print("The task is running. It has been running for {} hours and {} minutes.".format(round(hours,1), round(minutes,1)))

# If the task has finished
elif task_status["state"] == "COMPLETED":
  # Print a message saying that the task has finished
  print("The task has finished.")

# If the task is in an unknown state
else:
  # Print a message saying that the task is in an unknown state
  print("The task is in an unknown state.")

## View Aqua MODIS LAI

In [ ]:
# Create a folium/gee map object.
my_map = folium.Map(location=[34,265], zoom_start=5, height=1000) # Location is [lat,long]
Map8b = geemap.Map(center=[34,265], zoom=5, height=1000)

############Aqua MODIS MYD15A2H LAI############
def findLai(i):
  iveg = i.select('Lai_500m').multiply(0.1)
  return iveg

laiFun = myd15.filterDate(dateStartD, dateEndD).map(findLai)
laiFunMed = ee.ImageCollection(laiFun).median()

# Viewing the imagery
#my_map.add_ee_layer(laiFunMed.clip(multiReg),{'palette':['ff0000','ffff00','00ff00']},'LAI median')
#my_map.add_ee_layer(laiFun,{'min':0.19,'max':0.32},'LAI raw')
#my_map.add_child(folium.LayerControl())
#display(my_map)

# Adding the imagery to the interactive panel
Map8b.addLayer(laiFunMed.clip(prov),{'min':0,'max':3,'palette':['ff0000','ffff00','00ff00']},'LAI median')
#Map8b.addLayer(laiFun,{'min':0.19,'max':0.32},'LAI raw')
Map8b.addLayerControl()
Map8b

### Calculate LAI on Crop Mask

In [ ]:
# Create a folium/gee map object.
my_map = folium.Map(location=[34,265], zoom_start=5, height=1000) # location is [lat,long]
Map8aa = geemap.Map(center=[34,265], zoom=5, height=1000)

############Crop mask calculations############
mask = ee.Image(corn) # Crop mask here or CDL crop layer
maskFloat = mask.divide(100) # convert crop mask to float

def findLaiOnMask(i):
  lai = i.select('Lai_500m').multiply(0.1) # extract the correct layer and apply the scaling factor
  laiOnMa = lai.multiply(maskFloat)

  joinFilter = ee.Filter.equals(
  leftField='shapeID',
  rightField='shapeID'
  )

  # Get summed LAI * floating mask per region per date
  summedLOnM = laiOnMa.reduceRegions(
      collection=prov, # add the shapefile here
      reducer=ee.Reducer.sum(),
      scale=100
  ).select(**{
    'propertySelectors': ['shapeGroup','shapeName','shapeID','sum'],
    'newProperties': ['shapeGroup','shapeName','shapeID','sumLOnM']
    })

  # Get summed floating mask per region per date
  summedMask = maskFloat.reduceRegions(
      collection=prov, # add the shapefile here
      reducer=ee.Reducer.sum(),
      scale=100
  ).select(**{
    'propertySelectors': ['shapeID','sum'],
    'newProperties': ['shapeID','sumMask']
    })

  # Applying the join and perform the calculations over each region
  joinHere = ee.Join.inner()
  joinedJoin = joinHere.apply(summedLOnM,summedMask,joinFilter)
  def calcDone(i):
      getProp = ee.Feature(i.get('primary')).copyProperties(i.get('secondary'))
      return getProp.set({'calcedVal': ee.Number(getProp.get('sumLOnM')).divide(ee.Number(getProp.get('sumMask')))}) # divide the sums
  calculatedDistricts = joinedJoin.map(calcDone)

  return calculatedDistricts

laiOnMaFun = myd15.filterDate(dateStartD, dateEndD).map(findLaiOnMask).flatten()
laiOnMaFunMed = ee.ImageCollection(laiOnMaFun).median()

# Viewing the imagery
#my_map.add_ee_layer(laiFunMed.clip(multiReg),{'palette':['ff0000','ffff00','00ff00']},'LAI median')
#my_map.add_ee_layer(laiFun,{'min':0.19,'max':0.32},'LAI raw')
#my_map.add_child(folium.LayerControl())
#display(my_map)

# Adding the imagery to the interactive panel
#Map8aa.addLayer(mask.clip(multiReg),{'min':0,'max':100,},'Crop mask')
#Map8aa.addLayer(maskFloat.clip(multiReg),{'min':0,'max':1,},'Crop mask Float')
#Map8aa.addLayer(laiFunMed.clip(multiReg),{'min':0,'max':1},'LAI Median')
Map8aa.addLayer(laiOnMaFun,{},'LAI on Mask')
Map8aa.addLayerControl()
Map8aa

#print(eviOnMaFun.getInfo())

#### Export LAI on Crop Mask

In [ ]:
# Export the statistics
taskExL = ee.batch.Export.table.toDrive(**{
    'collection':laiOnMaFun,
    'folder':'Zonal_Stats', # you need to put the name of a folder from your Google Drive here
    'description':'LAIonCorn_season_202', # name of the exported file
    'fileFormat':'CSV',
    'selectors':['system:index','shapeGroup','shapeName','shapeID','sumLOnM','sumMask','calcedVal'] # columns
})

taskExL.start()

In [ ]:
# Check the task status
task_status = taskExL.status()

# If the task is running
if task_status["state"] == "RUNNING":
  start_time = task_status["start_timestamp_ms"]/(60000)
  update_time = task_status['update_timestamp_ms']/(60000)
  elapsed_time = update_time-(start_time)
  hours, minutes = divmod(elapsed_time, 60)
  # Print a message saying that the task is running and how long it has been running
  print("The task is running. It has been running for {} hours and {} minutes.".format(round(hours,1), round(minutes,1)))

# If the task has finished
elif task_status["state"] == "COMPLETED":
  # Print a message saying that the task has finished
  print("The task has finished.")

# If the task is in an unknown state
else:
  # Print a message saying that the task is in an unknown state
  print("The task is in an unknown state.")